In [1]:
from keras.models import Model
from keras.layers import Input, Conv2D, BatchNormalization, Activation, LeakyReLU, Add, Dense

Using TensorFlow backend.


In [2]:
class AnimeDiscriminatorFactory(object):
    """
        Discriminator Factory Class that creates the model described here:
        https://arxiv.org/pdf/1708.05509.pdf
    """
    
    def build(self, input_shape):
        """
            Returns a Model described here: https://arxiv.org/pdf/1708.05509.pdf
            
            Args:
                input_same: A 3 length tuple describing (width, height, channel)
                
            Output:
                Keras Model
        """
        RESIDUAL_BLOCKS_PER_LAYER = 2
        LEAKY_RELU_ALPHA = 0.2
        MODULES = 5
        
        def intermediate_layer(layer, filters, kernel_size):
            """
                Create the intermediate layers between residual layers.
                
                Args:
                    layer:       Keras layer
                    filters:     output size as an integer
                    kernel_size: length 2 tuple
                
                Returns:
                    Keras layer
            """
            layer = Conv2D(filters=filters, kernel_size=kernel_size, strides=(2, 2), padding="same")(layer)
            layer = LeakyReLU(alpha=LEAKY_RELU_ALPHA)(layer)
            return layer
        
        
        def initial_layer(input_layer):
            """
                Facade for intermediate_layer for the first layer of the network.
                 
                Args:
                    input_layer: Keras Input Layer
                
                Returns:
                    Keras layer
            """
            INITIAL_LAYER_FILTER = 32
            INITIAL_KERNEL_SIZE = (4, 4)
            return intermediate_layer(input_layer, INITIAL_LAYER_FILTER, INITIAL_KERNEL_SIZE)
        
#         def common_layer(layer):
#             layer = BatchNormalization()(layer)
#             layer = LeakyReLU(alpha=LEAKY_RELU_ALPHA)(layer)
#             return layer
        
        
        def residual_block(layer, filters):
            """
                Residual Block consisting of
                    Conv2D -> LeakyReLU -> Conv2D -> LeakyReLU -> Residual Addition
                
                Args:
                    layer:   Keras Layer
                    filters: output size as an integer
                
                Returns:
                    Keras layer
            """
            shortcut = layer
            layer = Conv2D(filters=filters, kernel_size=(3, 3), strides=(1, 1), padding="same")(layer)
            layer = LeakyReLU(alpha=LEAKY_RELU_ALPHA)(layer)
            layer = Conv2D(filters=filters, kernel_size=(3, 3), strides=(1, 1), padding="same")(layer)
            
            layer = Add()([layer, shortcut])
            layer = LeakyReLU(alpha=LEAKY_RELU_ALPHA)(layer)
            return layer
        
        
        def residual_layer(layer, number, filters):
            """
                Facade for residual block.
                
                Creates Residual layer with specified number of residual blocks
                
                Args:
                    layer:   Keras layer
                    number:  number of residual blocks in layer
                    filters: output size as an integer
                
                Returns:
                    Keras layer
            """
            for _ in range(number):
                layer = residual_block(layer, filters)
            return layer
        
                
        # NOTE: notation kxnysz
        # - k specifies that the convolution layer has kernel_size x
        # - n specifies that the convolution layer has y feature maps
        # - s specifies that the convolution layer has stride z
        
        inputs = Input(shape=input_shape)

        filters = 32
        # initial layer k4n32s2
        layer = initial_layer(inputs)
        for i in range(MODULES):
            layer = residual_layer(layer, RESIDUAL_BLOCKS_PER_LAYER, filters)
            filters *= 2
            
            intermediate_kernel_size = (3, 3)
            if i < 2:
                intermediate_kernel_size = (4, 4)
            layer = intermediate_layer(layer, filters, intermediate_kernel_size)
            
        outputs = Dense(2, activation="sigmoid")(layer)
        return Model(inputs=inputs, outputs=outputs)
        
        # What the loop should be doing.
#         # 2 residual blocks (0)
#         # A residual block is 2 * k3n32s1
#         layer = residual_layer(layer, RESIDUAL_BLOCKS_PER_LAYER, filters)
        
#         # intermediate layer k4n64s2
#         filters *= 2
#         layer = intermediate_layer(layer, filters, (4, 4))
        
#         # 2 residual blocks k3n64s1 (1)
#         layer = residual_layer(layer, RESIDUAL_BLOCKS_PER_LAYER, filters)
        
#         # intermediate layer k4n128s2
#         layer = intermediate_layer(layer, 128, (4, 4))
        
#         # 2 residual block k3n128s1 (2)
#         filters *= 2
#         layer = residual_layer(layer, RESIDUAL_BLOCKS_PER_LAYER, filters)
        
#         # intermediate layer k3n256s2
#         layer = intermediate_layer(layer, 256, (3, 3))
        
#         # 2 residual block k3n256s1 (3)
#         filters *= 2
#         residual_layer(layer, RESIDUAL_BLOCKS_PER_LAYER, filters)
        
#         # intermediate layer k3n512s2
#         layer = intermediate_layer(layer, 512, (3, 3))
        
#         # 2 residual block k3n512s1 (4)
#         filters *= 2
#         residual_layer(layer, RESIDUAL_BLOCKS_PER_LAYER, filters)
        
#         # intermediate layer k3n1024s2
#         layer = intermediate_layer(layer, 1024, (3, 3))
        
#         outputs = Dense(2, activation="sigmoid")(layer)      
#         return Model(inputs=inputs, outputs=outputs)
        

In [3]:
expected_anime_image = (64, 64, 3)
discriminator_factory = AnimeDiscriminatorFactory()
model = discriminator_factory.build(expected_anime_image)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 64, 64, 3)     0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 32, 32, 32)    1568        input_1[0][0]                    
____________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)        (None, 32, 32, 32)    0           conv2d_1[0][0]                   
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 32, 32, 32)    9248        leaky_re_lu_1[0][0]              
___________________________________________________________________________________________